# Tien Chau shoreline detection using `CoastSat`

This web notebook is based on a template provided by K. Vos et al. (2019).
The software `CoastSat` is described in details in the following publications: 
- Shoreline detection:                      https://doi.org/10.1016/j.envsoft.2019.104528
- Accuracy assessment and applications:     https://doi.org/10.1016/j.coastaleng.2019.04.004
- Beach slope estimation:                   https://doi.org/10.1029/2020GL088365

It enables the users to extract time-series of shoreline change over the last 30+ years at their site of interest.
There are four main steps:
1. Retrieval of the satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution
3. Intersection of the shorelines with cross-shore transects
4. Tidal correction 

## Initial settings

It is necessary to install the Anaconda system for Python, with necessary packages, including the Google Earth Engine Python API. 

Most likely you have to get a recent version of `matplotlibrc` file, by doing so: 
* navigate to the link https://raw.githubusercontent.com/matplotlib/matplotlib/v3.3.1/matplotlibrc.template
* save the file to your disk with name `matplotlibrc` (without extension)
* copy to the "matplotlib library folder", the exact folder is shown in the error message -- if any -- when you execute the code below (on my computer, the folder is `C:\Users\HP\.julia\conda\3\envs\coastsat\Lib\site-packages\matplotlib\mpl-data`)

Then the intialization is as following:

In [2]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

## 1. Retrieval of the images from GEE

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

Make sure the area of your ROI is smaller than 100 km<sup>2</sup> (if larger, split it into smaller ROIs).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided into Tier 1 and Tier 2. Only Tier 1 images can be used for time-series analysis. 

In [4]:
# region of interest (longitude, latitude)
polygon = [[[109.249, 13.3677],
            [109.266, 13.3677],
            [109.266, 13.3532],
            [109.249, 13.3532],
            [109.249, 13.3677]]]   

dates = ['1988-01-01', '2020-09-01']    # date range
sat_list = ['S2']   # satellite missions
sitename = 'TIENCHAU'   # name of the site
filepath = os.path.join(os.getcwd(), 'data')    # directory where the data will be stored

# put all the inputs into a dictionary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list,
        'sitename': sitename, 'filepath':filepath}

# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs)

Images available between 1988-01-01 and 2020-09-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 264 images
  Total: 264 images


The function `SDS_download.retrieve_images(inputs)` retrives the satellite images from Google Earth Engine.

By default, only Landsat Tier 1 Top-of-Atmosphere and Sentinel-2 Level-1C products are downloaded. 

In case you need to access Tier 2 images for qualitative analysis, you need to set `inputs['include_T2'] = True` before calling `retrieve_images`.

In [5]:
# inputs['include_T2'] = True
metadata = SDS_download.retrieve_images(inputs)

Images available between 1988-01-01 and 2020-09-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 264 images
  Total: 264 images

S2: 264 images
100%
23 out of 264 Sentinel-2 images were merged (overlapping or duplicate)


**If you have already retrieved the images**, just load the metadata file by only running the section below

In [6]:
metadata = SDS_download.get_metadata(inputs) 

## 2. Shoreline extraction

This section maps the position of the shoreline on the satellite images. The user can define the cloud threhold (`cloud_thresh`) and select the spatial reference system in which to output the coordinates of the mapped shorelines (`output_epsg`). See http://spatialreference.org/ to find the EPSG number corresponding to your local coordinate system. Make sure that your are using cartesian coordinates and not spherical coordinates (lat,lon) like WGS84. If unsure, use 3857 which is the web mercator projection (used by Google Maps).

To quality control each shoreline detection and manually validate the mapped shorelines, the user has the option to set the parameter `check_detection` to **True**. To save a figure for each mapped shoreline set `save_figure` to **True**. 

The other parameters are for advanced users only and are described in the README.

In [7]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

### [OPTIONAL] Save .jpg of the satellite images 
Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

In [8]:
SDS_preprocess.save_jpg(metadata, settings)

Satellite images saved as .jpg in D:\NQChien\CoastSat-master\data\TIENCHAU\jpg_files\preprocessed


### [OPTIONAL] Digitize a reference shoreline
Creates a reference shoreline which helps to identify outliers and false detections. The reference shoreline is manually digitised by the user on one of the images. The parameter `max_dist_ref` defines the maximum distance from the reference shoreline (in metres) at which a valid detected shoreline can be. If you think that the default value of 100 m will not capture the full shoreline variability of your site, increase this value to an appropriate distance.

For the case of Tien Chau it is necessary to add a reference shoreline because of the complex shape of the shoreline.

In [9]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100  # max distance (in meters) allowed from the reference shoreline

Reference shoreline has been saved in D:\NQChien\CoastSat-master\data\TIENCHAU


### Batch shoreline detection
Extracts the 2D shorelines from the images in the spatial reference system specified by the user in `'output_epsg'`. The mapped shorelines are saved into `output.pkl` (under *./data/sitename*) and `output.geojson` (to be used in a GIS software).

If you see that the sand pixels on the images are not being identified, change the parameter `sand_color` from `default` to `dark` or `bright` depending on the color of your beach. 

Most image data are available only from 2017 up to now. Many earlier images are covered with clouds, or the resolutions were not adequate.

In [10]:
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:
S2:   100%


Then remove duplicates and images with inaccurate georeferencing (threhsold at 10m)

In [11]:
output = SDS_tools.remove_duplicates(output)  # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10)  # remove inaccurate georeferencing (set threshold to 10 m)

1 duplicates
1 bad georef


In [21]:
len(output)

7

For use in GIS applications, you can save the mapped shorelines as a GEOJSON layer which can be easily imported into QGIS for example. You can choose to save the shorelines as a collection of lines or points (sometimes the lines are crossing over so better to use points).

In [12]:
geomtype = 'lines'  # choose 'points' or 'lines' for the layer geometry
gdf = SDS_tools.output_to_gdf(output, geomtype)
gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])}  # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'], '%s_output_%s.geojson'%(sitename,geomtype)),
                                driver='GeoJSON', encoding='utf-8')

Copies of the output files `TIENCHAU_*.pkl` and `TIENCHAU_*.geojson` have been made.

Simple plot of the mapped shorelines. The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover.

In [13]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()

The image has been saved to `All_shorelines.png` in `TIENCHAU`.

## 3. Shoreline analysis

In this section we show how to compute time-series of cross-shore distance along user-defined shore-normal transects.

**If you have already mapped the shorelines**, just load the output file (`output.pkl`) by running the section below

In [14]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f) 

There are 3 options to define the coordinates of the shore-normal transects:

**Option 1**: (method of choice - but remember to save the transect locations) the user can interactively draw the shore-normal transects along the beach by calling:

In [ ]:
%matplotlib qt
transects = SDS_transects.draw_transects(output, settings)

In [17]:
print(transects)

{'1': array([[12162127.3485389 ,  1500992.00673831],
       [12162208.58686935,  1501057.6223129 ]]), '2': array([[12162474.17371887,  1500720.17078645],
       [12162517.91743526,  1500788.9109122 ]]), '3': array([[12162727.26236371,  1500610.81149546],
       [12162749.13422191,  1500704.54803059]]), '4': array([[12162952.23004802,  1500551.44502322],
       [12162967.85280387,  1500682.67617239]]), '5': array([[12163017.8456226 ,  1500532.69771619],
       [12163242.81330691,  1500451.45938575]]), '6': array([[12163017.8456226 ,  1500517.07496034],
       [12163005.34741792,  1500417.08932287]]), '7': array([[12162805.37614298,  1500554.56957439],
       [12162824.12345001,  1500395.21746467]]), '8': array([[12162342.94256969,  1500623.30970015],
       [12162277.32699511,  1500567.06777907]])}


**Option 2**: the user can load the transect coordinates (make sure the spatial reference system is the same as defined previously by the parameter *output_epsg*) from a .geojson file by calling:

In [ ]:
geojson_file = os.path.join(os.getcwd(), 'examples', 'NARRA_transects.geojson')
transects = SDS_tools.transects_from_geojson(geojson_file)

**Option 3**: manually provide the coordinates of the transects as shown in the example below:

In [ ]:
transects = dict([])
transects['NA1'] = np.array([[16843142, -3989358], [16843457, -3989535]])
transects['NA2'] = np.array([[16842958, -3989834], [16843286, -3989983]])
transects['NA3'] = np.array([[16842602, -3990878], [16842955, -3990949]])
transects['NA4'] = np.array([[16842596, -3991929], [16842955, -3991895]])
transects['NA5'] = np.array([[16842838, -3992900], [16843155, -3992727]])

Plot the location of the transects, make sure they are in the right location with the origin always landwards!

In [18]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects.keys())):
    plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
    plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
    plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
                va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))

Now, intersect the transects with the 2D shorelines to obtain time-series of cross-shore distance.

The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 

In [19]:
# defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25 
cross_distance = SDS_transects.compute_intersection(output, transects, settings) 

Time-series of the shoreline change along the transects saved as:
D:\NQChien\CoastSat-master\data\TIENCHAU\transect_time_series.csv


Plot the time-series of shoreline change along each transect

In [20]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-50,50])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)  

## 4. Tidal correction

This last section is meant to tidally-correct the time-series of shoreline change using time-series of tide level and an estimate of the beach slope.

The estimated water levels for Phu Yen coast are retrieved from WxTide32 program and stored in the csv file `tide.csv`. In this file the times are in UTC, and the datum for the water levels is approx. Mean Sea Level.

Based on the measured topography, we choose a typical beach slope to be 0.05 along all transects.

In [36]:
output2 = output.copy()
output2.keys()

dict_keys(['dates', 'shorelines', 'filename', 'cloud_cover', 'geoaccuracy', 'idx', 'satname'])

In [39]:
output2['dates'][0]
o2 = {'dates':[output2['dates'][0]], 
      'shorelines':[output2['shorelines'][0]], 
      'filename':[output2['filename'][0]],
      'cloud_cover':[output2['cloud_cover'][0]],
      'geoaccuracy':[output2['geoaccuracy'][0]],
      'idx':[output2['idx'][0]],
      'satname':[output2['satname'][0]],
     }

In [ ]:
output2['filename'][:5], output2['filename'][-5:]

In [50]:
# load the measured tide data
filepath = os.path.join(os.getcwd(),'data','TIENCHAU','TIENCHAU_tides.csv')
tide_data = pd.read_csv(filepath, parse_dates=['dates'])
dates_ts = [_.to_pydatetime() for _ in tide_data['dates']]
tides_ts = np.array(tide_data['tide'])

# get tide levels corresponding to the time of image acquisition
dates_sat = output['dates']  #o2[...]
tides_sat = SDS_tools.get_closest_datapoint(dates_sat, dates_ts, tides_ts)

# plot the subsampled tide data
fig, ax = plt.subplots(1,1,figsize=(15,4), tight_layout=True)
ax.grid(which='major', linestyle=':', color='0.5')
ax.plot(tide_data['dates'], tide_data['tide'], '-', color='0.6', label='all time-series')
ax.plot(dates_sat, tides_sat, '-o', color='k', ms=6, mfc='w',lw=1, label='image acquisition')
ax.set(ylabel='tide level [m]',xlim=[dates_sat[0],dates_sat[-1]], title='Water levels at the time of image acquisition');
ax.legend()


Extracting closest points: 0%
Extracting closest points: 1%
Extracting closest points: 2%
Extracting closest points: 3%
Extracting closest points: 4%
Extracting closest points: 5%
Extracting closest points: 6%
Extracting closest points: 7%
Extracting closest points: 8%
Extracting closest points: 9%
Extracting closest points: 10%
Extracting closest points: 11%
Extracting closest points: 12%
Extracting closest points: 13%
Extracting closest points: 14%
Extracting closest points: 15%
Extracting closest points: 16%
Extracting closest points: 17%
Extracting closest points: 18%
Extracting closest points: 19%
Extracting closest points: 20%
Extracting closest points: 21%
Extracting closest points: 22%
Extracting closest points: 23%
Extracting closest points: 24%
Extracting closest points: 25%
Extracting closest points: 26%
Extracting closest points: 27%
Extracting closest points: 28%
Extracting closest points: 29%
Extracting closest points: 30%
Extracting closest points: 31%
Extracting closes

Apply tidal correction using a linear slope and a reference elevation to which project all the time-series of cross-shore change (to get time-series at Mean Sea Level, set `reference elevation` to 1.182 (the mean water level obtained from the time series). For the beach slope, a value of 0.05 is used for all transects.

In [ ]:
# tidal correction along each transect
reference_elevation = 1.182 # = average of tidal time series, elevation at which you would like the shoreline time-series to be
beach_slope = {'1':0.05, '2':0.05, '3':0.05, '4':0.05, '5':0.05, '6':0.05, '7':0.05, '8':0.05}
cross_distance_tidally_corrected = {}
for key in cross_distance.keys():
    correction = (tides_sat - reference_elevation)/beach_slope[key]
    cross_distance_tidally_corrected[key] = cross_distance[key] + correction
    
# store the tidally-corrected time-series in a .csv file
out_dict = dict([])
out_dict['dates'] = dates_sat
for key in cross_distance_tidally_corrected.keys():
    out_dict['Transect '+ key] = cross_distance_tidally_corrected[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'transect_time_series_tidally_corrected.csv')
df.to_csv(fn, sep=',')
print('Tidally-corrected time-series of the shoreline change along the transects saved as:\n%s'%fn)

In [74]:
# plot the time-series of shoreline change (both raw and tidally-corrected)
fig = plt.figure(figsize=[8,5], tight_layout=True)
# gs = gridspec.GridSpec(len(cross_distance),1)
gs = gridspec.GridSpec(2,1) # for 2 subplots only 
# gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05) # original
gs.update(left=0.1, right=0.9, bottom=0.1, top=0.9, hspace=0.1)
count = -1 # counting subplots
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    if key!='3' and key!='5':
        continue  # only choose to plot for transects 3 and 5
    count += 1
    ax = fig.add_subplot(gs[count,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-150,150])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=3, mfc='w', label='raw')
    ax.plot(output['dates'], cross_distance_tidally_corrected[key]- np.nanmedian(cross_distance[key]), '-o', ms=3, mfc='w', label='tidally-corrected')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, 'Transect '+key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='center', transform=ax.transAxes, fontsize=8)
ax.legend()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [55]:
len(out_dict['Transect 1'])

102

In [62]:
cross_distance.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8'])

## Remarks

After tidal correction, the cross-shore distance is `cross_distance_tidally_corrected[key] - np.nanmedian(cross_distance[key]`. Of which, the transect with key `'5'` locates approximately at the tip of the sand spit. 

The length of the sand spit is approximately the alongshore distance from transect 3 to 5. 

The area of the sand spit is approximately bounded by the transects 3 and 7.